In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
#importamos los ratings para cada lugar
Y_data = pd.read_csv('ratings_place.csv', sep=",", header = None).values
Y_data.shape

(34, 57)

In [3]:
#importamos la lista de lugares
places_list = pd.read_csv('locations.csv', sep=",", skiprows =[0], usecols=[1], header = None).values
places_list.shape

(34, 1)

In [4]:
#Logica del filtrado colaborativo para el minado de reglas de asociacion

Y = tf.placeholder(tf.float32)

X = tf.Variable(tf.truncated_normal([34, 100], stddev=0.1))
Theta = tf.Variable(tf.truncated_normal([57, 100], stddev=0.1))

#####
R = tf.cast(tf.not_equal(Y, 0), tf.float32)

Y_mean = (tf.reduce_sum(Y, axis = 1) / tf.reduce_sum(R, axis = 1))

Y_norm = Y - tf.reshape(Y_mean,[-1,1])

####

cost = 0.5 * tf.reduce_sum(R * (tf.square(tf.matmul(X, tf.transpose(Theta)) - Y_norm)))

train_step = tf.train.AdamOptimizer(1e-3).minimize(cost)

#predicciones
prediction = tf.matmul(X, tf.transpose(Theta)) + tf.reshape(Y_mean,[-1,1])

#creamos la sesion de tf
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

#entrenamiento
for i in range(10000):
    if i % 100 == 0:
        print('step %d, training cost %g' % (i, cost.eval(feed_dict={Y: Y_data})))
    sess.run(train_step, feed_dict={Y: Y_data})

Y_pred = prediction.eval(feed_dict={Y: Y_data})
print('Predicted Y:\n', Y_pred)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
step 0, training cost 577.65
step 100, training cost 178.496
step 200, training cost 21.4377
step 300, training cost 1.4448
step 400, training cost 0.11945
step 500, training cost 0.0127455
step 600, training cost 0.00150139
step 700, training cost 0.000174441
step 800, training cost 1.86843e-05
step 900, training cost 1.78307e-06
step 1000, training cost 1.49439e-07
step 1100, training cost 1.12309e-08
step 1200, training cost 9.58165e-10
step 1300, training cost 3.91474e-10
step 1400, training cost 6.24276e-11
step 1500, training cost 1.37967e-10
step 1600, training cost 2.61361e-11
step 1700, training cost 2.07584e-08
step 1800, training cost 1.57056e-11
step 1900, training cost 1.80303e-09
step 2000, training cost 8.85878e-12
step 2100, training cost 2.17089e-09
step 2200, training cost 6.29429e-12
step 2300, training cost 5.93317e-12
step 2400, training cost 2.45

In [5]:

recommend = np.argsort(Y_pred, axis=0)


# Muestra las k mejores recomendaciones para el usuario j
k = 10
j = 0

for i in range (k):
    print(places_list[recommend[i,j],0])


Museo Centro Cultural Manta
El faro
Malecon Escenico Murcielago
Monumento Al Atun Manta
Playa San Mateo
Playa Barbasquillo
Playa La Resbalosa La Piñas y Santa Rosa
Playa El Murciélago
Monumento a la Tejedora Manabita
Parque central


In [6]:
#ahora cargamos los perfiles por usuario en un df

In [7]:
sess.close()

In [8]:
visual = pd.DataFrame(recommend)
visual

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,4,4,1,15,1,4,16,16,1,2,...,25,25,25,25,25,25,25,25,25,25
1,16,2,27,1,15,2,3,1,21,1,...,32,29,29,29,29,29,29,29,29,29
2,1,1,17,6,30,1,1,6,18,6,...,29,33,31,33,33,33,33,31,33,33
3,14,11,5,0,25,6,8,21,17,9,...,22,31,33,31,31,31,32,33,31,31
4,10,15,24,18,4,0,23,18,31,28,...,31,32,32,32,32,32,31,32,32,32
5,9,16,25,28,2,30,24,17,13,5,...,33,24,24,24,24,24,24,24,24,5
6,13,6,2,32,16,23,25,5,5,31,...,12,5,22,22,5,5,22,22,5,24
7,2,3,4,24,3,22,4,29,24,33,...,24,12,12,12,12,12,12,12,12,12
8,15,26,15,2,21,24,2,24,25,25,...,5,13,5,5,28,13,13,5,22,13
9,3,14,3,4,26,31,15,25,4,29,...,13,22,13,13,13,22,5,13,13,22


In [9]:
np.save('train_matrix.npy', recommend)
rec_tensor = tf.stack(recommend)

In [20]:
import tensorflow.lite as lite
from tensorflow.lite.toco.python import toco_from_protos

users = pd.read_csv('user.csv', sep=",", header = None, skiprows=[0], usecols=[1,2,3]).values

inp = tf.placeholder(name="inp", dtype=tf.float32, shape=(1, 3))

tuser = tf.Variable(users, dtype=tf.float32)

tplace = tf.stack(recommend)

tplace = tf.transpose(tplace)

normed_inp = tf.nn.l2_normalize(inp, axis=1)
normed_users = tf.nn.l2_normalize(tuser, axis=1)

cosine_similarity = tf.matmul(normed_users, tf.transpose(normed_inp, [1, 0]))

closest = tf.argmax(cosine_similarity, 0)

places = tf.gather(tplace, closest)


init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    output = sess.run(places, feed_dict={inp: [[20,1,1]]})
    print(output)
    
    
    #converter = lite.TFLiteConverter.from_session(sess, [inp], [places])
    #tflite_model = converter.convert()
    #open("recsys.tflite", "wb").write(tflite_model)


[[ 1 15 30 25  4  2 16  3 21 26 11 10 27  8 28 32 12 33 13 24  9  6 14  0
  18  7 19 17 22 20 31  5 29 23]]
